## Sentiment Analysis on IMDB movies reviews
IMDB dataset having 50K movie reviews for natural language processing or Text analytics.
This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training and 25,000 for testing. So, predict the number of positive and negative reviews using either classification or deep learning algorithms.

In [3]:
### install tensorflow GPU
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
##check the version of tensorflow
import tensorflow as tf
print(tf.__version__)

2.8.2


In [5]:
import pandas as pd

In [6]:
##read the CSV file 
df=pd.read_csv('/content/drive/MyDrive/IMDB Dataset.csv')
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [7]:
df.shape

(50000, 2)

In [8]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [9]:
##Get the independent features
x=df.drop('sentiment', axis=1)

In [10]:
x

,review
0,One of the other reviewers has mentioned that ...
1,A wonderful little production. <br /><br />The...
2,I thought this was a wonderful way to spend ti...
3,Basically there's a family where a little boy ...
4,"Petter Mattei's ""Love in the Time of Money"" is..."
...,...
49995,I thought this movie did a down right good job...
49996,"Bad plot, bad dialogue, bad acting, idiotic di..."
49997,I am a Catholic taught in parochial elementary...
49998,I'm going to have to disagree with the previou...


In [11]:
##Get the dependent variable
df['sentiment']

0        positive
1        positive
2        positive
3        negative
4        positive
           ...   
49995    positive
49996    negative
49997    negative
49998    negative
49999    negative
Name: sentiment, Length: 50000, dtype: object

In [12]:
## feature Engineering
df["sentiment"].replace('positive',1,inplace=True)
df["sentiment"].replace('negative',0,inplace=True)

In [13]:
y=df['sentiment']

In [14]:
y

0        1
1        1
2        1
3        0
4        1
        ..
49995    1
49996    0
49997    0
49998    0
49999    0
Name: sentiment, Length: 50000, dtype: int64

In [15]:
import nltk
import re
from nltk.corpus import stopwords

In [16]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [17]:
###corpus cleaning
from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0, len(x)):
    review = re.sub('[^a-zA-Z]', ' ', x['review'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [18]:
## import important libraries
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [19]:
voc_size=15000
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
#onehot_repr

In [ ]:
sent_length=350
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
#print(embedded_docs)

In [21]:
## Creating model
embedding_vector_features=40 ##features representation
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 350, 40)           600000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 656,501
Trainable params: 656,501
Non-trainable params: 0
_________________________________________________________________
None


In [22]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [23]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [24]:
### Early Stopping(if the accuracy of training dataset is not increasing then stop training)
import tensorflow as tf
early_stopping=tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0.0001,
    patience=20,
    verbose=1,
    mode="auto",
    baseline=None,
    restore_best_weights=False,
)

In [25]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=100,batch_size=64,callbacks=early_stopping)

Epoch 1/100
524/524 [==============================] - 257s 485ms/step - loss: 0.6932 - accuracy: 0.5056 - val_loss: 0.6932 - val_accuracy: 0.5029
Epoch 2/100
524/524 [==============================] - 252s 481ms/step - loss: 0.6901 - accuracy: 0.5051 - val_loss: 0.6923 - val_accuracy: 0.5042
Epoch 3/100
524/524 [==============================] - 255s 486ms/step - loss: 0.6778 - accuracy: 0.5147 - val_loss: 0.6941 - val_accuracy: 0.5063
Epoch 4/100
524/524 [==============================] - 259s 494ms/step - loss: 0.6702 - accuracy: 0.5242 - val_loss: 0.7008 - val_accuracy: 0.5050
Epoch 5/100
524/524 [==============================] - 257s 490ms/step - loss: 0.6669 - accuracy: 0.5213 - val_loss: 0.7023 - val_accuracy: 0.5062
Epoch 6/100
524/524 [==============================] - 252s 480ms/step - loss: 0.6717 - accuracy: 0.5253 - val_loss: 0.6973 - val_accuracy: 0.5056
Epoch 7/100
524/524 [==============================] - 250s 478ms/step - loss: 0.6664 - accuracy: 0.5246 - val_loss: 0

In [26]:
y_pred=model.predict(X_test)

In [27]:
y_pred=np.where(y_pred > 0.6, 1,0) ##AUC ROC Curve

In [28]:
from sklearn.metrics import confusion_matrix

In [29]:
confusion_matrix(y_test,y_pred)

array([[6828, 1380],
       [1039, 7253]])

In [30]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.8533939393939394

In [31]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.87      0.83      0.85      8208
           1       0.84      0.87      0.86      8292

    accuracy                           0.85     16500
   macro avg       0.85      0.85      0.85     16500
weighted avg       0.85      0.85      0.85     16500

